In [ ]:
import re
import pandas as pd
from collections import Counter
import ast
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
from collections import defaultdict
from tqdm.notebook import tqdm

from utils import check_options, extract_option

PATH = "/home/cc/PHD/HealthBranches/"
BENCH =  f"{PATH}results/results_QUIZ_bench_"
BASE =  f"{PATH}results/results_QUIZ_baseline_"
MEDQA =  f"{PATH}results-medqa/results_QUIZ_medqa_"

In [ ]:
old = pd.read_csv(f"{PATH}questions_pro/dataset_updated.csv")
new = pd.read_csv(f"{PATH}refined_paths.csv")

In [ ]:
len(old['question'].unique()), len(new['question'].unique()), len(old), len(new)
old = old.drop_duplicates(subset=['question'])
new = new.drop_duplicates(subset=['question'])

print(len(old))
print(len(new))

old.to_csv(f"{PATH}questions_pro/dataset_updated_nodup.csv", index=False)
new.to_csv(f"{PATH}questions_pro/dataset_updated_V2path.csv", index=False)

In [ ]:
# Carica i CSV
df_dataset = pd.read_csv('/home/cc/PHD/HealthBranches/questions_pro/ultimate_questions_v3_full_balanced.csv')       # CSV completo delle domande
df_subset = pd.read_csv('/home/cc/PHD/HealthBranches/questions_to_check.csv')         # CSV con le domande da correggere
df_corrected = pd.read_csv('/home/cc/PHD/HealthBranches/question_checked.csv')    # CSV con le domande corrette; contiene la colonna "ID" (l'indice nel CSV subset)

# Resetta l'indice di df_subset per garantire che sia numerato da 0 a len(df_subset)-1
# df_subset.reset_index(drop=True, inplace=True)

mapping = {}
for _, row in df_corrected.iterrows():
    idx = int(row['ID'])
    if idx < len(df_subset):
        original_question = df_subset.loc[idx, 'question']
        mapping[original_question] = {
            'question': row['question'],
            'answer': row['answer'],
            'options': row['options'],
            'correct_option': row['correct_option'],
            'path': row['path']
        }
    else:
        print(f"Warning: ID {idx} non presente in df_subset")

# Funzione per aggiornare una riga del dataset, se la domanda è presente nel mapping
def update_row(row):
    corrections = mapping.get(row['question'])
    if corrections:
        row['question'] = corrections['question']
        row['answer'] = corrections['answer']
        row['options'] = corrections['options']
        row['correct_option'] = corrections['correct_option']
        row['path'] = corrections['path']
    return row

# Applica la funzione a ogni riga del dataset
df_dataset = df_dataset.apply(update_row, axis=1)
df_dataset.drop(columns=['answer'], inplace=True)

# Salva il dataset aggiornato
df_dataset.to_csv('dataset_updated.csv', index=False)
print("Dataset aggiornato salvato come 'dataset_updated.csv'")